In [2]:
%load_ext autoreload
%autoreload 2

import os, sys
from pathlib import Path

from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

from pathlib import Path

### Load a classifier

In [400]:
from collab_splats.utils.grouping import GroupingClassifier, GroupingConfig

# Path to the config for a trained model
load_config = '/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-25_074037/config.yml'
load_config = Path(load_config)

grouping_config = GroupingConfig(segmentation_backend='mobilesamv2', segmentation_strategy='object', front_percentage=0.2, iou_threshold=0.1, num_patches=32)
grouping_classifier = GroupingClassifier(load_config=load_config, config=grouping_config)

Memory bank loaded from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/grouping/memory_bank.pkl with 548 masks


### Associate masks

In [426]:
grouping_classifier.create_masks()

Loading train ordered data...


Creating masks [train]: 100%|██████████| 441/441 [00:00<00:00, 921.73it/s]


In [427]:
grouping_classifier.associate()

Loading train ordered data...


Processing frames:  50%|████▉     | 220/441 [00:03<00:03, 59.12it/s]

No objects found in frame_00232.png
No objects found in frame_00235.png


Processing frames: 100%|██████████| 441/441 [00:07<00:00, 57.93it/s]


Setup for running training

In [167]:
grouping_classifier.setup()

In [260]:
# Grab training batch
camera, batch = grouping_classifier.pipeline.datamanager.next_train(None)

# Get default outputs (just to check that things work)
# test = grouping_classifier.model.get_outputs(camera)

Get logits

In [ ]:
# Pass through custom render
identities = grouping_classifier._render_identities(camera)

# Get logits
grouping_classifier.classifier(identities.permute(2, 0, 1))

### Try pytorch lightning datamodule

In [449]:
from collab_splats.utils.grouping import GroupingDataModule 

datamodule = GroupingDataModule(
    datamanager=grouping_classifier.pipeline.datamanager,
    mask_dir=grouping_classifier.associated_mask_dir,
    device="cuda",
    train_num_workers=0,
    val_num_workers=0,
)